# Import Libraries

In [25]:
import os
import string
import torch
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from datasets import Dataset, DatasetDict, Features, Value,ClassLabel
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.nn.functional import cross_entropy
from transformers import pipeline
from re import sub
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from transformers import Trainer
from utils.preprocessing import get_clean_dataset
os.environ['WANDB_DISABLED'] = 'true'

In [26]:
bs = 8 # batch size
lr = 3e-5
epochs = 1
w_decay = 0.01
num_labels = 34
# model_ckpt = "roberta-base"
# model_ckpt = "facebook/bart-base"
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Dataset

In [ ]:

# twitter_comments = pd.read_csv('/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/DataSet/csv/twitter_comments.csv', sep=";", encoding='cp1252')

# input_text = []

# print("#Removing unnecessary character from the user comments.....")
# df = get_clean_dataset(twitter_comments)
# for comnt in df['text']:
#     # print(comnt)
#     comnt = str(comnt)
#     # print(comnt)
#     # Can we set the input comment lenght which will be consider for sentiment analysis?
#     if len(comnt) > 5:
#         input_text.append(comnt)

# print("Done!")

In [27]:
# Reading the 3 different datasets from the CSV files
dataset1 = pd.read_csv('/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/DataSet/csv/tweet_emotions.csv')
dataset1 = dataset1.drop(columns='tweet_id')
dataset2 = pd.read_csv('/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/DataSet/csv/tweet_emotions_1.csv')
dataset3 = pd.read_csv('/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/DataSet/csv/reddit_emotions.csv')

# Concat the Datasets

In [28]:
# Concating the 3 dataset into single one
data_frame = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)
data_frame = get_clean_dataset(data_frame)
# data_frame['text'] = data_frame['text'].apply(lambda x: remove_unnecessary_char(x))


In [29]:
class_names = []
with open('/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/DataSet/emotions.txt') as f:
    while True:
        line = f.readline()
        if not line:
            break
        # print(line.strip())
        class_names.append(line.strip())

mapping = {}
for i, line in enumerate(class_names):
    mapping[line] = i

print(mapping)

{'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'boredom': 5, 'caring': 6, 'confusion': 7, 'curiosity': 8, 'desire': 9, 'disappointment': 10, 'disapproval': 11, 'disgust': 12, 'embarrassment': 13, 'excitement': 14, 'enthusiasm': 15, 'fear': 16, 'fun': 17, 'gratitude': 18, 'grief': 19, 'happiness': 20, 'hate': 21, 'joy': 22, 'love': 23, 'nervousness': 24, 'optimism': 25, 'pride': 26, 'realization': 27, 'relief': 28, 'remorse': 29, 'sadness': 30, 'surprise': 31, 'worry': 32, 'neutral': 33}


In [30]:
data_frame = data_frame.loc[data_frame["label"] != 'empty' ]
data_frame['label'] = data_frame['label'].str.strip().map(mapping)
data_frame.drop(data_frame.columns[-1], axis=1, inplace=True)
# data_frame['label'].value_counts()

In [31]:
data_frame = shuffle(data_frame)
train_size = 0.8
validate_size = 0.1
# Spliting the dataset into 3 parts Training, Test, and Validation
train, validate, test = np.split(data_frame, [int(train_size * len(data_frame)), int((validate_size + train_size) * len(data_frame))])

In [32]:
print('Dataset information:')
print(f'Training data: {train.shape}')
print(f'Validation data: {validate.shape}')
print(f'Test data: {test.shape}')

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
validate = test.reset_index(drop=True)

Dataset information:
Training data: (114571, 2)
Validation data: (14321, 2)
Test data: (14322, 2)


In [33]:
print(train['label'].unique())
print(test['label'].unique())
print(validate['label'].unique())

[ 4 33 30 22 11  8 14 32  3 24  2  7 16  0 27 25  5  1 13 10 18 31 23 20
 17 29 28 15 12  9 26  6 21 19]
[ 8 20 23 33  3  7 25 31 22  9 32  0 16 21 11 28 30  2 10  4  1  6 29 24
 14 27 18 19 17 12 13 15 26  5]
[ 8 20 23 33  3  7 25 31 22  9 32  0 16 21 11 28 30  2 10  4  1  6 29 24
 14 27 18 19 17 12 13 15 26  5]


In [34]:
train['label'].value_counts()

33    21889
30    10574
22     7658
32     6740
23     6541
4      4697
0      4545
2      4401
20     4174
31     3854
3      3701
18     3130
11     3104
16     2793
8      2566
1      2481
27     2463
25     2277
10     2262
7      1998
6      1608
28     1512
14     1471
17     1451
12     1449
9      1094
21     1055
29      680
13      638
15      618
24      498
26      328
19      178
5       143
Name: label, dtype: int64

In [35]:
import plotly.express as px
from collections import Counter
from matplotlib import pyplot
import plotly.express as px

# counter = Counter(train['label'])
# for k,v in counter.items():
#  per = v / len(train['label']) * 100
#  print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

# px.bar(df['label_name'].value_counts(ascending=True),template='plotly_white')

In [36]:
# {'admiration': 0, 'amusement': 1, 'anger': 2, 'annoyance': 3, 'approval': 4, 'boredom': 5, 'caring': 6, 'confusion': 7, 
# 'curiosity': 8, 'desire': 9, 'disappointment': 10, 'disapproval': 11, 'disgust': 12, 'embarrassment': 13, 'excitement': 14, 
# 'enthusiasm': 15, 'fear': 16, 'fun': 17, 'gratitude': 18, 'grief': 19, 'happiness': 20, 'hate': 21, 'joy': 22, 'love': 23, 
# 'nervousness': 24, 'optimism': 25, 'pride': 26, 'realization': 27, 'relief': 28, 'remorse': 29, 'sadness': 30, 'surprise': 31, 
# 'worry': 32, 'neutral': 33}

strategy = {0:5000, 1:3000, 3:4000, 5:1500, 6:2000, 7:2500, 8:3000, 9:1500, 10:2500, 11:3500, 12:1500, 13:1500, 14:2000, 
            15:1500, 16:3000, 17:1500, 18:4000, 19:1500, 21:1500, 24:1500, 25:2500, 26:1500, 27:3000, 28:2000, 29:1500}

In [37]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.datasets import make_imbalance
from sklearn.preprocessing import LabelEncoder

X = train['text']
y = train['label']

ros = RandomOverSampler(sampling_strategy=strategy) # sampling_strategy=strategy
# ros = SMOTE(sampling_strategy=strategy)

train_x, train_y = ros.fit_resample(np.array(X).reshape(-1, 1), np.array(y).reshape(-1, 1))
train = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'label'])
# train['label'].value_counts()

In [38]:
# Setting the feature
ft = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})

# Combine Multiple Datasets 
emotions = DatasetDict({
    "train": Dataset.from_pandas(train, features=ft),
    "test": Dataset.from_pandas(test,features=ft),
    "validation": Dataset.from_pandas(validate,features=ft)
    })

In [39]:
# Add label data to dataframe
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

In [40]:
# Convert Dataset to DataFrame (don't forget to reset)
emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

,text,label
0,report physical emotional abuse police record ...,4
1,American trade journalists ask PRs 100% time U...,33
2,finishing 91 minutes sitting half hour burstin...,33
3,think sick week got rut feel low like hard rou...,30
4,youtube working properly cant watch videos fai...,30


In [41]:
df["label"] = df["label"].apply(label_int2str)
px.bar(df["label"].value_counts(), template='plotly_white')

In [42]:
# Convert a single DataFrame to a Dataset
# emotions = Dataset.from_pandas(train,features=ft)
train_ds = emotions["train"]
# Load parameters of the tokeniser
# model_ckpt = "distilbert-base-uncased"
# model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# model = AutoModel.from_pretrained(model_ckpt)
# Model to device for accessing the GPU if supported otherthan that device would be CPU
# model.to(device)

print('#Info related to tokenizer!')
print(f'Vocab size: {tokenizer.vocab_size}')
print(f'Max length: {tokenizer.model_max_length}')
print(f'Tokeniser model input names: {tokenizer.model_input_names}')

#Info related to tokenizer!
Vocab size: 30522
Max length: 512
Tokeniser model input names: ['input_ids', 'attention_mask']


In [43]:
text = 'Tokenisation of text is a core task of NLP.'
print('Encoded text')
encoded_text = tokenizer(text)
print(encoded_text,'\n')
print('Tokens')
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens,'\n')

print('#Convert tokens to string')
print(tokenizer.convert_tokens_to_string(tokens),'\n')

Encoded text
{'input_ids': [101, 19204, 6648, 1997, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

Tokens
['[CLS]', 'token', '##isation', 'of', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '.', '[SEP]'] 

#Convert tokens to string
[CLS] tokenisation of text is a core task of nlp. [SEP] 



In [44]:
emotions.reset_format()

# Tokenisation function
def tokenise(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# apply to the entire dataset (train,test and validation dataset)
emotions_encoded = emotions.map(tokenise, batched=True, batch_size=None)
print(emotions_encoded["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

['text', 'label', 'input_ids', 'attention_mask']


In [ ]:
from torch.nn.parallel import DistributedDataParallel as DDP

emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)
# model = torch.nn.DataParallel(model, device_ids=[0, 1])
model.to(device)

In [49]:
# For Measureing Model performance
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [50]:
# using time module
import time
# ts stores the time in seconds
ts = time.time()

logging_steps = len(emotions_encoded["train"]) // bs
out_dir = "Trend-Prediction/models/"
model_name = f"{model_ckpt}-finetuned-emotion-" + str(ts)
log_dir = "/home/siu856533724/code/pc114/Social-Networks/Trend-Prediction/logging/runs/"+str(ts)
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=epochs,      # number of training epochs
                                  learning_rate=lr,             # model learning rate
                                  per_device_train_batch_size=bs, # batch size
                                  per_device_eval_batch_size=bs,  # batch size
                                  weight_decay=w_decay,           # weight decay
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False, 
                                  report_to="none",
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error"
                                  )


trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer,
                  )
trainer.train()

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=8065, training_loss=2.230902966134532, metrics={'train_runtime': 539.9421, 'train_samples_per_second': 238.966, 'train_steps_per_second': 14.937, 'total_flos': 8818093703498112.0, 'train_loss': 2.230902966134532, 'epoch': 1.0})

In [ ]:
# Predict on Validation Dataset
pred_output = trainer.predict(emotions_encoded["test"])

print(f'Output Predition: {pred_output.predictions.shape}')
print(pred_output.predictions)

y_preds = np.argmax(pred_output.predictions,axis=1)
print(f'Output Prediction:{y_preds.shape}')
print(f'Predictions: {y_preds}')

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                             reduction="none")
        
    # Place outputs on CPU for compatibility with other dataset columns
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

# Convert our dataset back to PyTorch tensors
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
# Compute loss values
emotions_encoded["test"] = emotions_encoded["test"].map(forward_pass_with_label,
                                                                    batched=True, 
                                                                    batch_size=bs)

In [ ]:
# Add label data to dataframe
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

In [ ]:
emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["test"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"].apply(label_int2str))

In [ ]:
df_test.sort_values("loss", ascending=True).head(10)

In [ ]:
trainer.save_model()
classifier = pipeline("text-classification", model=model_name)

In [ ]:
# New unseen by model data
new_data = "I watched a movie last night, it was quite brilliant"

preds = classifier(new_data, return_all_scores=True)
df_preds = pd.DataFrame(preds[0])
print("Input text:")
print(new_data)
print("Predicted Score:")
print(df_preds)

In [ ]:
import plotly.express as px
labels = emotions["train"].features["label"].names
px.bar(x=labels,y=100*df_preds['score'],template='plotly_white')